In [30]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
from pandas_ta.momentum import rsi as ta_rsi
import plotly.graph_objects as go


import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import HeikinAshi

from binance.client import Client

#Conexion con Binance sin API-KEY
client = Client()

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [31]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [32]:
klines_file  = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-10-15.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #df.set_index('datetime',inplace=True)
    
start = '2024-11-12'
end = '2024-11-13'
symbol = 'BTCUSDT'
interval = '5m'
real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
utc_start = real_start+' 00:00:00'
utc_end = real_end+' 23:59:59'
df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)
#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)
print(utc_start,utc_end)
df.tail()

BTCUSDT 5m 2024-11-12 00:00:00 2024-11-13 23:59:59
2024-11-12 00:00:00 2024-11-13 23:59:59


,datetime,open,high,low,close,volume
283,2024-11-12 20:35:00,87985.99,88036.95,87840.06,88036.94,93.54199
284,2024-11-12 20:40:00,88036.95,88038.75,87718.81,87881.29,90.24948
285,2024-11-12 20:45:00,87881.29,88028.31,87766.00,88000.00,208.61895
286,2024-11-12 20:50:00,87999.99,88172.00,87984.00,88144.00,278.69996
287,2024-11-12 20:55:00,88144.00,88177.43,88108.00,88154.14,41.18406


In [44]:
df_origin = df[:-100]
# Crear una copia para trabajar en Heikin-Ashi con el close promedio entre close anterior y actual
df = HeikinAshi(df)

df['trend-chg'] = np.where(df['HA_low_trend'],df['close'],None)

df['close'] = df['HA_close']
df['open'] = df['HA_open']
df['high'] = df['HA_high']
df['low'] = df['HA_low']

#df['HA_sl'] =  np.where((df['HA_side']>=0) & (df['HA_side'].shift(1)>=0),df['HA_low'].shift(2),None)
#df['HA_tp'] =  np.where((df['HA_side']<=0) & (df['HA_side'].shift(1)<=0),df['HA_high'].shift(2),None)
#
#df['HA_sl'].ffill(inplace=True)
#df['HA_tp'].ffill(inplace=True)

df['HA_sl'] =  df['HA_low'].shift(2)
df['HA_tp'] =  df['HA_high'].shift(2)
buy_cond = ( 
           (df['HA_sl']>df['HA_sl'].shift(1))  
           & (df['HA_sl'].shift(2)>df['HA_sl'].shift(1)) 
           & (df['HA_side'].shift(3)<=0) 
           & (df['HA_side'].shift(4)<=0) 
           & (df['HA_side'].shift(5)<=0) 
           & (df['HA_side'].shift(6)<=0) 
           )
#buy_cond = (df['HA_sl']>df['HA_tp']) & (df['HA_sl'].shift(1)<df['HA_tp'].shift(1)) & (df['HA_sl']!=df['HA_sl'].shift(1))
df['buy'] = np.where(buy_cond,df['close'],None)
df['sell'] = None


In [45]:
df_to_show = df #[0:20]
indicators = [
     {'col': 'HA_sl','name': 'HA_sl','color': 'red','row': 1, 'mode':'lines',},
     {'col': 'HA_tp','name': 'HA_tp','color': 'green','row': 1, 'mode':'lines',},
     #{'col': 'trend-chg','name': 'trend-chg','color': 'yellow','row': 1, 'mode':'markers',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
     #   {'col': 'FL_0.0%', 'name': 'FL_0.0%', 'color': 'red', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold1', 'name': 'FL_gold1', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold2', 'name': 'FL_gold2', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_100.0%', 'name': 'FL_100.0%', 'color': 'green', 'row': 1, 'mode': 'lines', }, 
        ]

indicators_out = [
     #{'col': 'sign_acum','name': 'sign_acum','color': 'yellow','row': 1, 'mode':'lines',},
     #{'col': 'VOL_perc','name': 'Volume_SMA','color': 'green','row': 1, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       {'df':df,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       {'df':df,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()